In [ ]:
#解压一下略小改之后的PaddleSeg，解压一次就可以注释掉了
!unzip -oq /home/aistudio/PaddleSeg.zip

In [1]:
#解压数据集至data/目录
!unzip -qo data/data95249/train_50k_mask.zip -d data/
!unzip -oq data/data100087/B榜测试数据集.zip -d data/
!unzip -oq data/data95249/train_image.zip -d data/

In [ ]:
import sys
sys.path.append("PaddleSeg")
import paddleseg
import paddle
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import random
#设置随机数种子
random.seed(2021)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


In [ ]:
def write_txt(file_name, imgs_path, labels_path=None, mode='train', val_pro=0.2):
    assert mode=="train" or mode=="test", "ERROR:mode must be train or test."
    if mode!="test":
        train_path = []
        for idx, f_path in enumerate(imgs_path):
            for i_path in sorted(os.listdir(f_path)):
                path1 = os.path.join(f_path, i_path) 
                path2 = os.path.join(labels_path[idx], i_path)
                train_path.append((path1, path2, str(idx)))
        
        if val_pro>=0 and val_pro<=1:
            #打乱数据
            random.shuffle(train_path)
            val_len = int(len(train_path)*val_pro)
            val_path = train_path[:val_len]
            train_path = train_path[val_len:]
            with open(file_name[0], 'w') as f:
                for path in train_path:
                    f.write(path[0]+" "+path[1]+" "+path[2]+"\n")
            with open(file_name[1], 'w') as f:
                for path in val_path:
                    f.write(path[0]+" "+path[1]+" "+path[2]+"\n")  
            return len(train_path), val_len
        else:
            with open(file_name[0], 'w') as f:
                for path in train_path:
                    f.write(path[0]+" "+path[1]+" "+path[2]+"\n") 
            return len(train_path), 0
    else:
        with open(file_name, 'w') as f:
            for path in imgs_path:
                img_path = os.path.join(test_path, path)
                f.write(img_path+"\n")

In [ ]:
def create_txt(data_root, train_imgs_dir=None, train_labels_dir=None, test_dir=None, val_pro=0.2):
    if train_imgs_dir is not None:
        if os.path.exists("train.txt"):
            os.remove("train.txt")
        if os.path.exists("val.txt"):
            os.remove("val.txt")
        train_imgs_dir = os.path.join(data_root, train_imgs_dir)
        train_labels_dir = os.path.join(data_root, train_labels_dir)
        file_names = os.listdir(train_imgs_dir)
        file_names = sorted(file_names)
        train_imgs_path, train_labels_path =[], []
        for na in file_names:
            train_imgs_path.append(os.path.join(train_imgs_dir, na))
            train_labels_path.append(os.path.join(train_labels_dir, na))
        train_len, val_len = write_txt(["train.txt", "val.txt"], train_imgs_path, train_labels_path, mode='train', val_pro=val_pro)
        
        print("训练数据整理完毕！训练集长度：{}，验证集长度：{}， 类别数：{}".format(train_len, val_len, len(file_names)))

    if test_dir is not None:
        if os.path.exists("test.txt"):
            os.remove("test.txt")
        global test_path
        test_path = os.path.join(data_root, test_dir)
        test_imgs_path_list = sorted(os.listdir(test_path))
        write_txt("test.txt", test_imgs_path_list, mode="test")
        print("测试数据整理完毕！测试集长度：{}".format(len(test_imgs_path_list)))

In [ ]:
data_root = "data"
train_imgs_dir = "train_image"
train_labels_dir = "train_50k_mask"
test_dir = "test_image"
create_txt(data_root, train_imgs_dir, train_labels_dir, test_dir, val_pro=0.2)

训练数据整理完毕！训练集长度：40000，验证集长度：10000， 类别数：500
测试数据整理完毕！测试集长度：10989


In [ ]:
#验证一下是否写入正确，可以直接点开文件查看
#也可以读取文件内容查看
#以train.txt为例，只看前5行验证
count = 5
with open('train.txt', 'r')  as f:
    for line in f.readlines():
        print(line)
        count -= 1
        if count==0:
            break

data/train_image/n02101006/n02101006_6140.png data/train_50k_mask/n02101006/n02101006_6140.png 138

data/train_image/n03124170/n03124170_6952.png data/train_50k_mask/n03124170/n03124170_6952.png 333

data/train_image/n04037443/n04037443_18517.png data/train_50k_mask/n04037443/n04037443_18517.png 448

data/train_image/n01770393/n01770393_2677.png data/train_50k_mask/n01770393/n01770393_2677.png 44

data/train_image/n07613480/n07613480_19014.png data/train_50k_mask/n07613480/n07613480_19014.png 485



In [2]:
# 训练模型
!python PaddleSeg/train.py --config my_deeplabv3.yml --do_eval --use_vdl --save_dir /home/aistudio/output_deeplabv3_1 --save_interval 2000

In [3]:
#推理
!python PaddleSeg/predict.py --config my_deeplabv3.yml --model_path output_deeplabv3_1/best_model/model.pdparams --image_path data/test_image --save_dir output/result_1 #--aug_pred --flip_horizontal --flip_vertical

2021-08-05 22:32:35 [INFO]	
---------------Config Information---------------
batch_size: 8
iters: 10000
loss:
  coef:
  - 1
  types:
  - coef:
    - 1.0
    losses:
    - type: DiceLoss
    type: MixedLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.001
  power: 0.9
  type: PolynomialDecay
model:
  align_corners: false
  aspp_out_channels: 256
  aspp_ratios:
  - 1
  - 6
  - 12
  - 18
  - 24
  backbone:
    multi_grid:
    - 1
    - 2
    - 4
    output_stride: 8
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/resnet101_vd_ssld.tar.gz
    type: ResNet101_vd
  backbone_indices:
  - 0
  - 3
  num_classes: 2
  pretrained: output_deeplabv3_1/best_model/model.pdparams
  type: DeepLabV3P
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: /home/aistudio
  mode: train
  num_classes: 2
  train_path: /home/aistudio/train.txt
  transforms:
  - type: RandomHorizontalFlip
  - type: RandomVerticalFlip
  - brightness_range: 0.4
    contrast_range: 0.

In [ ]:
import os
import cv2
import numpy as np
filePath = 'output/result_1'
filenames = os.listdir(filePath)
kernel = np.ones((3,3),np.uint8)
for filename in filenames:
    img = cv2.imread(filePath + filename)
    erosion = cv2.erode(img,kernel,iterations = 1)
    gaussian = cv2.GaussianBlur(erosion,(5, 5), 2)
    cv2.imwrite('./pred/' + filename, gaussian)

In [ ]:
%cd output/result_1/results
!zip -r -oq /home/aistudio/pred.zip ./
%cd /home/aistudio

/home/aistudio/output/result_1/results
